# Department of State Procurement Forecast

https://www.state.gov/procurement-forecast

In [2]:
%run notebooks/Setup.ipynb

import pandas
import numpy
import re

In [3]:
# list the files for each FY
directory_path = workspace_path.joinpath('data/state_gov_procurement_forecast/raw')
procurement_parts = list(directory_path.glob('*.xlsx'))

# see how to align the schemas2
for part in procurement_parts:
    df = pandas.read_excel(part)
    print(part.name)
    print(df.dtypes)
    print('\n')

FY2023Forecast_StateDept_2023-01-09.xlsx
Requirement Description                   object
Office Symbol                             object
New Requirement?                          object
NAICS-Codes                               object
Past Competition                          object
Past Set-Aside                            object
Incumbent Contractor                      object
Place of Performance                      object
Est. Value                                object
Length of Performance                     object
Target Award Quarter                      object
Point of Contact Name                     object
Point of Contact Email Address            object
Modified                          datetime64[ns]
Created                           datetime64[ns]
dtype: object


FY2019-Forecast-20181213_2.xlsx
Fiscal Year                                   object
Requirement Description                       object
Office Symbol                                 object
N=New \nR= Recomp

In [3]:
# stuff we want in the final dataset
final_columns = [
    'Fiscal Year',
    'Requirement Title',
    'Requirement Description',
    'Office Symbol',
    'New Requirement?',
    'Incumbent Contractor',
    'Past Competition',
    'Length of Performance',
    'Target Award Fiscal Year Quarter',
    'Estimated Value',
    'Place of Performance',
    'Point of Contact Name',
    'Facility Security Clearance',
    'NAICS-Codes'
]

# for each FY, figure out which columns are missing and should be remapped
dfs = {}
for part in procurement_parts:
    df = pandas.read_excel(part)
    dfs[part.name] = df
    missing_columns = set(final_columns) - set(df.columns)
    print(part.name)
    print(missing_columns)
    print('\n')

FY2023Forecast_StateDept_2023-01-09.xlsx
{'Fiscal Year', 'Facility Security Clearance', 'Estimated Value', 'Requirement Title', 'Target Award Fiscal Year Quarter'}


FY2019-Forecast-20181213_2.xlsx
{'Place of Performance', 'New Requirement?', 'Past Competition', 'Facility Security Clearance', 'Requirement Title', 'NAICS-Codes'}


FY24-Department-of-State-Procurement-Forecast-11-2-2023.xlsx
{'Requirement Title', 'Target Award Fiscal Year Quarter', 'Estimated Value'}


Forecast-2020-Dec3Rev2.xlsx
{'Place of Performance', 'New Requirement?', 'Past Competition', 'Facility Security Clearance', 'Requirement Title', 'NAICS-Codes'}


FY25-Procurement-Forecast.xlsx
{'Place of Performance', 'Target Award Fiscal Year Quarter', 'Point of Contact Name'}


FY2022Forecast_StateDept_2022-03-30.xlsx
{'Requirement Title', 'Target Award Fiscal Year Quarter', 'Facility Security Clearance', 'Estimated Value'}


20211115ForecastPublished.xlsx
{'Place of Performance', 'New Requirement?', 'Past Competition', 

In [4]:
# little cleanup before we remap columns

# FY2023Forecast_StateDept_2023-01-09.xlsx is missing Fiscal Year, add that
dfs['FY2023Forecast_StateDept_2023-01-09.xlsx']['Fiscal Year'] = 'FY23'

# FY25-Procurement-Forecast.xlsx has contact names broken up, combine first and last
dfs['FY25-Procurement-Forecast.xlsx']['Point of Contact Name'] = (
    dfs['FY25-Procurement-Forecast.xlsx']['Point of Contact First Name'] + ' ' +
    dfs['FY25-Procurement-Forecast.xlsx']['Point Of Contact Last Name']
)

In [5]:
# remap the columns so we can stick the datasets together
missing_columns = {
    'FY2023Forecast_StateDept_2023-01-09.xlsx': {
        'Estimated Value': 'Est. Value',
        'Target Award Fiscal Year Quarter': 'Target Award Quarter'
    },
    'FY2019-Forecast-20181213_2.xlsx': {
        'Place of Performance': 'Place of Performance if Outside US',
        'Past Competition': 'History of Award Category',
        'New Requirement?': 'N=New \nR= Recompete',
        'NAICS-Codes': 'NAICS Code'
    },
    'FY24-Department-of-State-Procurement-Forecast-11-2-2023.xlsx': {
        'Target Award Fiscal Year Quarter': 'Target Award Quarter',
        'Estimated Value': 'EstimatedValue'
    },
    'Forecast-2020-Dec3Rev2.xlsx': {
        'Place of Performance': 'Place of Performance if Outside US',
        'Past Competition': 'History of Award Category',
        'New Requirement?': 'N=New \nR= Recompete',
        'NAICS-Codes': 'NAICS Code'
    },
    'FY25-Procurement-Forecast.xlsx': {
        'Target Award Fiscal Year Quarter': 'Fiscal Year',
        'Point of Contact Name': 'Point of Contact Name',
        'Place of Performance': 'Place of Performance Country'
    },
    'FY2022Forecast_StateDept_2022-03-30.xlsx': {
        'Target Award Fiscal Year Quarter': 'Target Award Quarter',
        'Estimated Value': 'EstimatedValue'
    },
    '20211115ForecastPublished.xlsx': {
        'Place of Performance': 'Place of Performance if Outside US',
        'Past Competition': 'History of Award Category',
        'New Requirement?': 'N=New \nR= Recompete',
        'NAICS-Codes': 'NAICS Code'
    }
}

for part, column_map in missing_columns.items():
    for column, mapped_column in column_map.items():
        dfs[part][column] = dfs[part][mapped_column]
    missing_columns = set(final_columns) - set(dfs[part].columns)

    # quick sanity check the missing columns are expected since the data doesn't totally align
    print(part)
    print(missing_columns)
    print('\n')

FY2023Forecast_StateDept_2023-01-09.xlsx
{'Requirement Title', 'Facility Security Clearance'}


FY2019-Forecast-20181213_2.xlsx
{'Requirement Title', 'Facility Security Clearance'}


FY24-Department-of-State-Procurement-Forecast-11-2-2023.xlsx
{'Requirement Title'}


Forecast-2020-Dec3Rev2.xlsx
{'Requirement Title', 'Facility Security Clearance'}


FY25-Procurement-Forecast.xlsx
set()


FY2022Forecast_StateDept_2022-03-30.xlsx
{'Requirement Title', 'Facility Security Clearance'}


20211115ForecastPublished.xlsx
{'Requirement Title', 'Facility Security Clearance'}




In [6]:
# combine all the parts, keeping only the specified columns
combined_df = pandas.concat(dfs.values(), ignore_index=True)[final_columns]

# rename some columns to play well in data warehouses
combined_df.rename(columns={
    'New Requirement?': 'New Requirement',
    'NAICS-Codes': 'NAICS Code'
}, inplace=True)

# uppercase description and title so we don't have to worry about case
combined_df['Requirement Title'] = combined_df['Requirement Title'].str.upper()
combined_df['Requirement Description'] = combined_df['Requirement Description'].str.upper()

# normalize the data across New Requirement
combined_df['New Requirement'] = combined_df['New Requirement'].replace({'R': 'Recompete', 'N': 'New'})

# normalize some popular values in Place of Performance
combined_df['Place of Performance'] = combined_df['Place of Performance'].str.upper().replace({
    # replace empty and null to 'US' since a couple nad "Place of Performance if Outside US"
    '': 'USA',
    None: 'USA',
    # normalize some stuff
    'US': 'USA',
    'UNITED STATES': 'USA',
    'UNITED STATES OF AMERICA': 'USA',
    'WASHINGTON DC': 'WASHINGTON, DC',
    'WASHINGTON DC - SOME TELEWORK': 'WASHINGTON, DC',
    'WASHINGTON,DC': 'WASHINGTON, DC',
    'WASHINGTON, DC.': 'WASHINGTON, DC',
    'WASHINGTON, D.C.': 'WASHINGTON, DC',
})

# clean up Fiscal Year data, e.g. FY 22 -> FY22
combined_df['Fiscal Year'] = combined_df['Fiscal Year'].str.replace('FY ', 'FY')

# Estimated value is either in form [">$1M and <$5M", "$1M-$5M", ">$250K and <$500K"], split into a lower and upper bound as separate columns and convert to numeric
# @o3-generated
def parse_estimated_value(val_str):
    if pandas.isnull(val_str):
        return numpy.nan, numpy.nan

    val_str = val_str.upper()

    # This regex will capture an optional '>' or '<', the dollar sign,
    # the numeric part (integer or float), and an optional multiplier (M or K).
    pattern = r'([><]?)\$?\s*(\d+(?:\.\d+)?)([MK]?)'
    parts = re.findall(pattern, val_str)

    # Helper function to convert the numeric string to a number with proper multiplier
    def convert(num, mult):
        num = float(num)
        if mult == 'M':
            return num * 1e6
        elif mult == 'K':
            return num * 1e3
        else:
            return num

    # Initialize lower and upper bounds as NaN
    lower = numpy.nan
    upper = numpy.nan

    # If there are two parts, assume the first is the lower bound and the second is the upper bound.
    if len(parts) == 2:
        lower = convert(parts[0][1], parts[0][2])
        upper = convert(parts[1][1], parts[1][2])
    # If only one part is found, decide based on the sign.
    elif len(parts) == 1:
        sign, num, mult = parts[0]
        val = convert(num, mult)
        if sign == '<':
            upper = val
        else:
            lower = val
    return lower, upper

# clean up Estimated Value into numerics
combined_df[['Estimated Value Lower', 'Estimated Value Upper']] = combined_df['Estimated Value'].apply(
    lambda x: pandas.Series(parse_estimated_value(x))
)

# cast some stuff to strings
combined_df['NAICS Code'] = combined_df['NAICS Code'].astype(str)

combined_df

,Fiscal Year,Requirement Title,Requirement Description,Office Symbol,New Requirement,Incumbent Contractor,Past Competition,Length of Performance,Target Award Fiscal Year Quarter,Estimated Value,Place of Performance,Point of Contact Name,Facility Security Clearance,NAICS Code,Estimated Value Lower,Estimated Value Upper
0,FY23,NaN,OVER THE PHONE TRANSLATION,CA/OCS,Recompete,Cyracom,Competitive,5 years,Q1,$1M-$5M,USA,"Williams, Denise A",NaN,541930 - Translation and Interpretation Services,1000000.0,5000000.0
1,FY23,NaN,ARCHITECTURE AND SPACE PLANNING CONSULTING,CA/EX/GSD,Recompete,ARCHVIA,Competitive,5 years,Q2,$5M-$10M,USA,"Kayani, Bilal A",NaN,541310 - Architectural Services,5000000.0,10000000.0
2,FY23,NaN,QUALTRICS SOFTWARE,CA/C,New,NaN,NaN,5 years,Q3,$500K-$1M,USA,"Rush, Noelle L",NaN,511210 - Software Publishers,500000.0,1000000.0
3,FY23,NaN,TOMIS - OFM (THE OFFICE OF FOREIGN MISSIONS IN...,OFM/OPS/IS,Recompete,Alpha Omega Integration LLC,Competitive,5 years,Q3,$1M-$5M,USA,"Douglas, Clayton F",NaN,519190 - All Other Information Services,1000000.0,5000000.0
4,FY23,NaN,MOBILE TELEPHONE COMMUNICATIONS,P/WHA/LIMA/GSO,Recompete,Telefonica del Peru S.A.A.,Competitive,5 years,Q1,$250k-500K,PERU,"Quimpo, Carlos A",NaN,517210 - Wireless Telecommunications Carriers ...,250000.0,500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3254,FY21,NaN,PROCUREMENT OF SLACK LICENSES,R/PPR,Recompete,Carahsoft,Competitive - Full and Open,Base Option Period (1yr),Quarter 4 (Jul-Sep),>$250K and <$500K,USA,Sheila Campbell,NaN,423430 - Computer and Computer Peripheral Equi...,250000.0,500000.0
3255,FY21,NaN,IT SUPPORT,WHA/EX/ADM,Recompete,RAVENTEK,Veteran Owned Small Business Competed,Base + 2 Option Periods (3yrs),Quarter 3 (Apr-Jun),>$250K and <$500K,USA,GUY R. MITCHELL,NaN,541519 - Other Computer Related Services,250000.0,500000.0
3256,FY21,NaN,IT SUPPORT,WHA/EX/ADM,Recompete,RAVENTEK,Service Disabled Veteran Owned Small Business ...,Base + 2 Option Periods (3yrs),Quarter 2 (Jan-Mar),>$250K and <$500K,USA,GUY R MITCHELL,NaN,541519 - Other Computer Related Services,250000.0,500000.0
3257,FY21,NaN,GRANT COORDINATOR/PROGRAM ADVISOR,WHA/EX/ADM,Recompete,Cherokee Nation,Certified HUBZone Small Business Competed,Base + 2 Option Periods (3yrs),Quarter 4 (Jul-Sep),>$150K and <$500K,USA,GUY R MITCHELL,NaN,561210 - Facilities Support Services,150000.0,500000.0


In [ ]:
# write to parquet and csv
combined_df.to_parquet(workspace_path.joinpath('data/state_gov_procurement_forecast/state_gov_procurement_forecast.parquet'))
combined_df.to_csv(workspace_path.joinpath('data/state_gov_procurement_forecast/state_gov_procurement_forecast.csv'), index=False)